In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import math
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import os

os.environ['MKP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
#
# Load train files
# 
train_fn = []

data = []
# Load all train files
for i in range(len(train_fn)):
    name = train_fn[i]
    matFile = loadmat('./emo_data/train' + name + '.mat')
    matFile.keys()
    data.append(matFile['data'])
    
# Note the second dimension for every file has a different length
data = np.array(data)
np.random.shuffle(data)
print(data.shape)

(0,)


In [3]:
# X coordinates of 1st person 22 body joints
X_coords = data[0][:, 0:22]
# Y coordinates of 1st person 22 body joints
Y_coords = data[0][:, 22:44]
# Z coordinates of 1st person 22 body joints
Z_coords = data[0][:, 44:66]

print(X_coords.shape)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [4]:
# Plot a single block of coordinate data
# Input: data (NxM)
def plot_single_line_coord_block(data):
    dims = data.shape[1]
    print('DIMS ', dims)
    for i in range(dims):
        plt.plot(data[:, i])
        
    return plt.show()

# Plot all three coordinate blocks together
def plot_all_coords(X, Y, Z):
    dims = X.shape[1]
    for i in range(dims):
        plt.plot(X[:, i], color='blue')
        plt.plot(Y[:, i], color='green')
        plt.plot(Z[:, i], color='red')
        
    plt.xlabel('Frames')
    plt.ylabel('Coordindate Position')
    plt.title('Coordinate data (Blue - X, Green - Y, Red - Z)')
    return plt.show()

In [5]:
plot_all_coords(X_coords, Y_coords, Z_coords)

NameError: name 'X_coords' is not defined

In [6]:
def format_single_person_coords(single_data_item):
    SW_LENGTH = int(180 - (180 * 0.5)) # 3 seconds + 50% overlap
    new_data = []
    new_data_labels = []
    new_data_labels_pain_intensity = []

    count = 0
    start = 0
    end = start + SW_LENGTH
    T = single_data_item.shape[0]

    while start < T:
        currentCoords = single_data_item[start:end, 0:70]
        
        # Majority vote the pain intensity level
        # NOTE: We add one to all the pain intensity levels
        # {0: Not Reported, 1: Healthy, 2: Low-Level pain, 3: High-Level Pain}
        pain = np.array(single_data_item[start:end, 71]).astype('int64') + 1
        majority_pain_intensity = np.argmax(np.bincount(pain))
        
        # Majority vote the behaviour
        behaviours = np.array(single_data_item[start:end, 72]).astype('int64')
        majority_protective_behaviour = np.argmax(np.bincount(behaviours))
        
        A, B = currentCoords.shape
        if A < 180:
            # We need to pad
            currentCoords = np.pad(currentCoords, ((0, 180 - A), (0, 0)))

        new_data.append(currentCoords)
        new_data_labels.append(majority_protective_behaviour)
        new_data_labels_pain_intensity.append(majority_pain_intensity)
        
        # Update pointers
        count += 1
        start += SW_LENGTH
        end = start + SW_LENGTH

    new_data = np.array(new_data)
    new_data_labels = np.array(new_data_labels)
    # print(new_data.shape)
    # print(new_data_labels.shape)
    
    return new_data, new_data_labels, new_data_labels_pain_intensity

# Example
new_data, new_data_labels, new_data_labels_pain = format_single_person_coords(data[9])

IndexError: index 9 is out of bounds for axis 0 with size 0

In [7]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
class ProtectiveModel(nn.Module):
    def __init__(self, input_size = 70, hidden_layer_size = 1024, output_size = 2):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers = 2)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size),
                            torch.zeros(1, 1, self.hidden_layer_size))
        
        # MLP
        self.linear1 = nn.Linear(hidden_layer_size, (hidden_layer_size // 2))
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear((hidden_layer_size // 2), output_size)
        
    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        lstm_out = lstm_out.view(len(input_seq), -1)
        
        protective_prediction = self.linear1(lstm_out.view(len(input_seq), -1))
        protective_prediction = self.relu(protective_prediction)
        protective_prediction = self.linear2(protective_prediction)
        
        return protective_prediction
    
class IntensityModel(nn.Module):
    def __init__(self, input_size = 70, hidden_layer_size = 1024, output_size = 4):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers = 2)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size),
                            torch.zeros(1, 1, self.hidden_layer_size))
        
        # MLP
        self.linear1 = nn.Linear(hidden_layer_size, (hidden_layer_size // 2))
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear((hidden_layer_size // 2), output_size)
        
    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        lstm_out = lstm_out.view(len(input_seq), -1)
        
        intensity_prediction = self.linear1(lstm_out.view(len(input_seq), -1))
        
        return intensity_prediction

In [9]:
#
# Handle the core train method
#
def train(loss, optim, epochs, model, data):
    for i in range(data.shape[0]):
        # Format
        new_data, new_data_labels, new_data_labels_pain = format_single_person_coords(data[i])
    
        for e in range(epochs):
            for j in range(new_daa.shape[0]):
                optim.zero_grad()
                model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size), torch.zeros(1, 1, model.hidden_layer_size))
                
                # Turn into something pytorch cares about
                data = torch.Tensor(new_data[j])
                label = torch.Tensor([new_data_labels[j]])
                pain = torch.Tensor([new_data_labels_pain[j]])
                pain = pain.repeat(180).long()
                
                out = model(data)
                
                # NOTE: This pain value needs to be changed accordingly to
                # what is being tested
                _loss = loss(out, pain)
                loss.backward()
                
                optim.step()
                
    return model

In [11]:
test_fn = []
test_data = []
# Load all train files
for i in range(len(test_fn)):
    name = test_fn[i]
    matFile = loadmat('./emo_data/validation/' + name + '.mat')
    matFile.keys()
    test_data.append(matFile['data'])
    
test_data = np.array(test_data)

In [ ]:
def test_model(model, data):
    total_percentage = 0
    total_intensity_percentage = 0
    
    for j in range(data.shape[0]):
        test_new_data, test_new_data_labels, test_new_data_pain = format_single_person_coords(data[j])

        count = 0
        intensity_count = 0
        
        for i in range(data.shape[0]):
            _data = torch.Tensor(data[i])
            _label = torch.Tensor([data[i]])
            _intensity = torch.Tensor([data[i]])

            pred = model(_data)
            pred = pred.max(1, keepdim=True)[1]

            # count += torch.sum(pred == _label).item()
            pred_value = np.argmax(np.bincount(pred.detach().numpy()[:, 0].astype('int64')))

            # NOTE: This value needs to be changed accordingly to what
            # model is being tested
            if (_label == pred_value):
                count += 1

        total_percentage += count / (test_new_data.shape[0]) * 100
        
    return total_percentage